EP5 - MAP2212 - 2024

Victor Rocha Cardoso Cruz, NUSP: 11223757

Larissa Aparecida Marques Pimenta Santos, NUSP: 12558620

ENUNCIADO:

- Reenvie o seu EP 4, substituindo o amostrador exato da distribuição Dirichlet por um amostrador Markov Chain Monte Carlo (MCMC) baseado apenas no conhecimento de um potencial para a distribuição Dirichlet.

- Use como núcleo para o MCMC uma normal multivariada N(0,Σ)
, onde a matriz de covariância é ajustada baseada em seu conhecimento prévio dos parâmetros da Dirichlet e/ou algumas sequências de amostragem iniciais ou de teste.

- No seu relatório, inclua o critério de diagnóstico utilizado para avaliar o progresso do MCMC, para ajustar o núcleo do MCMC, para decidir o número de pontos usados no aquecimento do núcleo do MCMC e para computar  W - a função verdade do modelo estatístico.

In [ ]:
# Código do EP 4

import numpy as np
import scipy
import time

# Variáveis globais utilizadas no programa
escolha_usuario = 0  # Variável para armazenar a escolha do usuário
vetor_x = []  # Vetor X
vetor_y = []  # Vetor Y
alpha = []  # Vetor alpha
dirichi = 0  # Variável para armazenar o resultado do potencial
contador = 0  # Contador utilizado em loops
auxiliar = 0  # Variável auxiliar
valores = []  # Lista para armazenar os valores do potencial
fronteira = []  # Lista para armazenar os valores fronteira dos grupos
quantidade = []  # Lista para armazenar os grupos de valores
beta = 0  # Constante de normalização
aproximacao = 0  # Valor de aproximação
V = 0  # Valor inserido pelo usuário

# Função para calcular o potencial
def potencial(dirichlet, alfa):
    return(dirichlet[0]**(alfa[0]-1))*(dirichlet[1]**(alfa[1]-1))*(dirichlet[2]**(alfa[2]-1))

# Vetores X e Y padrão
vetor_x.append(1)
vetor_x.append(2)
vetor_x.append(3)
vetor_y.append(4)
vetor_y.append(5)
vetor_y.append(6)

# Definindo alpha
alpha.append(vetor_y[0] + vetor_x[0])
alpha.append(vetor_y[1] + vetor_x[1])
alpha.append(vetor_y[2] + vetor_x[2])

# Calculando a constante de normalização beta
beta = (((scipy.special.gamma(alpha[0]))*(scipy.special.gamma(alpha[1]))*(scipy.special.gamma(alpha[2])))/(scipy.special.gamma(alpha[0]+alpha[1]+alpha[2])))

# Gerando os thetas Dirichlet
dirichlet = np.random.dirichlet(alpha, size=15375000)

# Calculando o potencial para cada vetor Dirichlet gerado e armazenando em uma lista
for contador in range(15375000):
    dirichi = (potencial(dirichlet[contador], alpha))
    valores.append(dirichi)
contador = 0

# Ordenando o vetor de valores do potencial
valores = sorted(valores)

# Dividindo os valores em grupos e armazenando em "quantidade" e o valor limite de cada grupo em "fronteira"
for contador in range(6150, 15375001, 6150):
    quantidade.append(valores[contador-6150:contador])
    fronteira.append(quantidade[auxiliar][6150-1])
    auxiliar += 1

# Solicitação ao usuário de um valor V
while True:
    print("Caso queira encerrar o programa insira um número menor que 0")
    V = float(input("Insira um valor de V (maior igual a zero): "))
    if V < 0:
        break
    else:
        k = np.searchsorted(fronteira, V*beta)
        # Quantidade de números até o corte
        aproximacao = 6150*k
        # Checagem do número interior ao corte
        if k != 2500:
            aproximacao += np.searchsorted(quantidade[k], V*beta)
        # Mostrando o valor de U
        resultado = aproximacao/15375000
        print("U(", V, ") = ", end="")
        print("%.4f\n" % resultado)

Caso queira encerrar o programa insira um número menor que 0
Insira um valor de V (maior igual a zero): 1
U( 1.0 ) = 0.0390

Caso queira encerrar o programa insira um número menor que 0
Insira um valor de V (maior igual a zero): 0.7
U( 0.7 ) = 0.0261

Caso queira encerrar o programa insira um número menor que 0
Insira um valor de V (maior igual a zero): 0.2
U( 0.2 ) = 0.0064

Caso queira encerrar o programa insira um número menor que 0
Insira um valor de V (maior igual a zero): 0.1
U( 0.1 ) = 0.0029

Caso queira encerrar o programa insira um número menor que 0
Insira um valor de V (maior igual a zero): -1


In [ ]:
# Código do EP 5

import numpy as np
import scipy.stats as stats
import time

# Definindo a função de potencial
def potencial(dirichlet, alpha):
    # Calcula o valor do potencial para uma amostra da distribuição Dirichlet
    multiplicatorio = (dirichlet[0]**(alpha[0]-1))*(dirichlet[1]**(alpha[1]-1))*(dirichlet[2]**(alpha[2]-1))
    return multiplicatorio

# Definindo a matriz de covariância 2D
def covar(alpha):
    # Calcula a matriz de covariância 2D para o kernel da distribuição normal multivariada
    vetor = scipy.stats.dirichlet.var(alpha)
    matriz = np.identity(2)
    soma = np.sum(alpha)
    for k in range(0, 2):
        for l in range(0, 2):
            if (k == l):
                matriz[k][k] = vetor[k]
            else:
                matriz[k][l] = -1*alpha[k]*alpha[l]/((soma**2)*(soma + 1))
    return matriz

# Função para amostrar usando MCMC com kernel normal multivariada
def distribu(pot, alpha, posicao, cov_matrix):
    contador = 0
    potencial_atual = pot
    potencial_prox = 0
    atual = posicao
    aceitos = []
    divi = 0
    auxiliar = 2
    ajuda = 0

    # Loop para gerar amostras usando MCMC
    while (contador != 3855000):
        # Gera amostras da distribuição normal multivariada
        normal = np.random.multivariate_normal([0, 0], cov=cov_matrix, size=3855000*auxiliar)
        ajuda = 0
        while (ajuda < 3855000*auxiliar and contador != 3855000):
            # Calcula o próximo ponto na cadeia de Markov
            proximo = [(atual[0]+normal[ajuda][0]), (atual[1]+normal[ajuda][1]), 1-(atual[0]+normal[ajuda][0]+atual[1]+normal[ajuda][1])]
            ajuda += 1
            # Calcula o potencial do próximo ponto
            potencial_prox = potencial(proximo, alpha)
            divi = potencial_prox/potencial_atual
            # Verifica se o próximo ponto é válido
            if(proximo[0] > 0 and proximo[1] > 0 and proximo[2] > 0):
                # Aceita o próximo ponto com base nas probabilidades
                if (divi < 1):
                    if(divi < np.random.uniform()):
                        pass
                    else:
                        contador += 1
                        if(contador > 5000):
                            aceitos.append(potencial_prox)
                        atual = proximo
                        potencial_atual = potencial_prox
                else:
                    contador += 1
                    if(contador > 5000):
                        aceitos.append(potencial_prox)
                    atual = proximo
                    potencial_atual = potencial_prox
        auxiliar = 1
    return aceitos

# Programa principal
vetor_x.append(1)
vetor_x.append(2)
vetor_x.append(3)
vetor_y.append(4)
vetor_y.append(5)
vetor_y.append(6)

# Definindo alpha e beta
alpha = [vetor_x[0] + vetor_y[0], vetor_x[1] + vetor_y[1], vetor_x[2] + vetor_y[2]]
beta = (scipy.special.gamma(alpha[0]) * scipy.special.gamma(alpha[1]) * scipy.special.gamma(alpha[2])) / scipy.special.gamma(np.sum(alpha))

# Calculando a matriz de covariância
cov_matrix = covar(alpha)

print("Gerando os valores usando MCMC\n")
start_time = time.time()
valores = distribu(potencial([1/3, 1/3, 1/3], alpha), alpha, [1/3, 1/3, 1/3], cov_matrix)
end_time = time.time()
tempo = end_time - start_time
print("O tempo gasto na geração dos valores é: %.1f segundos\n" % tempo)

# Ordenando os valores da função
valores = sorted(valores)

# Dividindo os valores em grupos
quantidade = []
fronteira = []
auxiliar = 0
for contador in range(1540, 3850001, 1540):
    quantidade.append(valores[contador-1540:contador])
    fronteira.append(quantidade[auxiliar][1540-1])
    auxiliar += 1

# Solicitando ao usuário um valor V
while True:
    print("Caso queira encerrar o programa, insira um número menor que 0")
    V = float(input("Insira um valor de V (maior igual a zero): "))
    if V < 0:
        break
    else:
        k = np.searchsorted(fronteira, V * beta)
        # Quantidade de números até o corte
        aproximacao = 1540 * k
        # Checagem do número interior ao corte
        if k != 2500:
            aproximacao += np.searchsorted(quantidade[k], V * beta)
        # Mostrando o valor de U
        resultado = aproximacao / 3850000
        print("U({}) = {:.4f}".format(V, resultado))

Gerando os valores usando MCMC

O tempo gasto na geração dos valores é: 39.6 segundos

Caso queira encerrar o programa, insira um número menor que 0
Insira um valor de V (maior igual a zero): 1
U(1.0) = 0.0360
Caso queira encerrar o programa, insira um número menor que 0
Insira um valor de V (maior igual a zero): 0.7
U(0.7) = 0.0238
Caso queira encerrar o programa, insira um número menor que 0
Insira um valor de V (maior igual a zero): 0.2
U(0.2) = 0.0057
Caso queira encerrar o programa, insira um número menor que 0
Insira um valor de V (maior igual a zero): 0.1
U(0.1) = 0.0026
Caso queira encerrar o programa, insira um número menor que 0
Insira um valor de V (maior igual a zero): -1
